# Supply Chain Shipment Pricing Prediction

## Background
In this study, analysis and pricing prediction is applied to supply chain shipment data. Models which will be explored during this analysis are listed and described below:

| Algorithm                                | Definition                                                                                                       | Characteristic                                                                                    |
|------------------------------------------|------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|
| Gradient boosted trees model (GBT Model) | Each tree is trained to predict and then "correct" for the errors of the previously trained trees                | A set of shallow decision trees trained sequentially.                                             |
| Multiple linear regression (MLR)         | A statistical technique for estimating a predictive target utilizing a linear relationship between two or more predictive factors for one predictive target. | Predicts a dependent variable using multiple independent variables.                               |
| Deep neural network (DNN)                | An artificial neural network consisting of many hidden layers between an input and output layer.                 | This algorithm can model complex nonlinear relationships, and it contains multiple hidden layers. |
| XGBoost regression                       | Extreme gradient boosting acting as an improved algorithm based on the gradient boosting algorithm.              | Excellent efficiency, flexibility, and portability, and it can prevent overfitting.               |
| LightGBM regression                      | Gradient boost-based algorithm that includes two techniques. (1) Gradient based on one-side sampling and (2) exclusive feature bundling. | An ensemble technique that utilizes a leaf-wise tree partitioning method.                         |

### Import Packages

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import math

In [2]:
print("Found TensorFlow Decision Forests v" + tfdf.__version__)

Found TensorFlow Decision Forests v1.3.0


## Data Collection and Preprocessing

### Data Collection
This data set provides supply chain health commodity shipment and pricing data. Specifically, the data set identifies Antiretroviral (ARV) and HIV lab shipments to supported countries. In addition, the data set provides the commodity pricing and associated supply chain expenses necessary to move the commodities to countries for use. The dataset has similar fields to the Global Fund's Price, Quality and Reporting (PQR) data. PEPFAR and the Global Fund represent the two largest procurers of HIV health commodities. This dataset, when analyzed in conjunction with the PQR data, provides a more complete picture of global spending on specific health commodities. The data are particularly valuable for understanding ranges and trends in pricing as well as volumes delivered by country. The US Government believes this data will help stakeholders make better, data-driven decisions. Care should be taken to consider contextual factors when using the database. Conclusions related to costs associated with moving specific line items or products to specific countries and lead times by product/country will not be accurate.


In [3]:
# Load the data set into Pandas DataFrame
dataset_df = pd.read_csv("Supply_Chain_Shipment_Pricing_Data.csv")

dataset_df.tail()

,id,project code,pq #,po / so #,asn/dn #,country,managed by,fulfill via,vendor inco term,shipment mode,...,unit of measure (per pack),line item quantity,line item value,pack price,unit price,manufacturing site,first line designation,weight (kilograms),freight cost (usd),line item insurance (usd)
10319,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",False,See DN-4307 (ID#:83920),See DN-4307 (ID#:83920),705.79
10320,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,...,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,False,See DN-4313 (ID#:83921),See DN-4313 (ID#:83921),161.71
10321,86821,110-ZM-T30,FPQ-14784,SO-49600,DN-4316,Zambia,PMO - US,From RDC,N/A - From RDC,Truck,...,30,514526,5140114.74,9.99,0.33,Cipla Ltd A-42 MIDC Mahar. IN,False,Weight Captured Separately,Freight Included in Commodity Cost,5284.04
10322,86822,200-ZW-T30,FPQ-16523,SO-51680,DN-4334,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,17465,113871.80,6.52,0.11,Mylan (formerly Matrix) Nashik,True,1392,Freight Included in Commodity Cost,134.03
10323,86823,103-ZW-T30,FPQ-15197,SO-50022,DN-4336,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,36639,72911.61,1.99,0.03,"Cipla, Goa, India",False,Weight Captured Separately,Freight Included in Commodity Cost,85.82


### Data Preprocessing

In [4]:
# Explore country count data before removing values.
CountryCount1 = dataset_df['country'].value_counts().nlargest(50)
print(CountryCount1)

country
South Africa          1406
Nigeria               1194
Côte d'Ivoire         1083
Uganda                 779
Vietnam                688
Zambia                 683
Haiti                  655
Mozambique             631
Zimbabwe               538
Tanzania               519
Rwanda                 430
Congo, DRC             333
Guyana                 237
Ethiopia               216
South Sudan            164
Kenya                  111
Burundi                 98
Namibia                 95
Cameroon                75
Botswana                70
Ghana                   58
Dominican Republic      52
Sudan                   46
Swaziland               35
Mali                    17
Guatemala               15
Pakistan                15
Malawi                  14
Benin                   13
Lebanon                  8
Libya                    8
Angola                   7
Liberia                  6
Sierra Leone             4
Lesotho                  4
Senegal                  3
Togo                

Country count values are represented greatly by South Africa, Nigeria, and Côte d'Ivoire with values over 1000.

In [5]:
# Describe numerical values before removal section
dataset_df.describe()

,id,unit of measure (per pack),line item quantity,line item value,pack price,unit price,line item insurance (usd)
count,10324.000000,10324.000000,10324.000000,1.032400e+04,10324.000000,10324.000000,10037.000000
mean,51098.968229,77.990895,18332.534870,1.576506e+05,21.910241,0.611701,240.117626
std,31944.332496,76.579764,40035.302961,3.452921e+05,45.609223,3.275808,500.190568
min,1.000000,1.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,12795.750000,30.000000,408.000000,4.314593e+03,4.120000,0.080000,6.510000
50%,57540.500000,60.000000,3000.000000,3.047147e+04,9.300000,0.160000,47.040000
75%,83648.250000,90.000000,17039.750000,1.664471e+05,23.592500,0.470000,252.400000
max,86823.000000,1000.000000,619999.000000,5.951990e+06,1345.640000,238.650000,7708.440000


In [6]:
# Describe manufacturing origin before removal section
ManLocs1 = dataset_df.groupby('manufacturing site').size().nlargest(5)
print(ManLocs1)

manufacturing site
Aurobindo Unit III, India         3172
Mylan (formerly Matrix) Nashik    1415
Hetero Unit III Hyderabad IN       869
Cipla, Goa, India                  665
Strides, Bangalore, India.         540
dtype: int64


Weight and freight cost are not included in this numerical description method because of the dtype caused by the inclusion of "value references".

#### Creating and Removing Variables
Removing `weight (kilograms)` values which are not directly referenced within the dataset.

In [7]:
dataset_df['weight (kilograms)'] = pd.to_numeric(dataset_df['weight (kilograms)'], errors = 'coerce')
dataset_df.dropna(inplace = True)

Removing `freight cost (usd)` values that are not referenced within the dataset. 

In [8]:
dataset_df['freight cost (usd)'] = pd.to_numeric(dataset_df['freight cost (usd)'], errors = 'coerce')
dataset_df.dropna(inplace = True)

Now let's see if the remaining dataset is prepared for further processing.

In [9]:
dim = dataset_df.shape

print(f"Shape of Shipment Pricing dataset after weight and cost reference removal: {dim}")

dataset_df.info()

Shape of Shipment Pricing dataset after weight and cost reference removal: (4571, 33)
<class 'pandas.core.frame.DataFrame'>
Index: 4571 entries, 18 to 10316
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            4571 non-null   int64  
 1   project code                  4571 non-null   object 
 2   pq #                          4571 non-null   object 
 3   po / so #                     4571 non-null   object 
 4   asn/dn #                      4571 non-null   object 
 5   country                       4571 non-null   object 
 6   managed by                    4571 non-null   object 
 7   fulfill via                   4571 non-null   object 
 8   vendor inco term              4571 non-null   object 
 9   shipment mode                 4571 non-null   object 
 10  pq first sent to client date  4571 non-null   object 
 11  po sent to vendor date        4571 non

We can focus on the statistics regarding the numerical columns of the dataset.

In [10]:
dataset_df.describe()

,id,unit of measure (per pack),line item quantity,line item value,pack price,unit price,weight (kilograms),freight cost (usd),line item insurance (usd)
count,4571.000000,4571.000000,4571.000000,4.571000e+03,4571.000000,4571.000000,4571.000000,4571.000000,4571.000000
mean,56957.054255,79.959090,27470.185955,2.091655e+05,12.019807,0.198324,3908.025159,10761.065629,318.840171
std,30959.704494,86.895991,48231.724706,4.169485e+05,18.385518,0.290996,7876.806664,15193.737445,609.247953
min,115.000000,5.000000,1.000000,0.000000e+00,0.000000,0.000000,1.000000,14.360000,0.000000
25%,22402.000000,30.000000,1591.000000,1.043055e+04,3.400000,0.060000,268.000000,2336.265000,16.075000
50%,75733.000000,60.000000,8676.000000,5.778000e+04,7.300000,0.140000,1347.000000,5900.000000,86.360000
75%,83868.500000,60.000000,33422.000000,2.247840e+05,11.750000,0.230000,4287.000000,13322.900000,343.335000
max,86815.000000,1000.000000,619999.000000,5.951990e+06,306.880000,14.040000,154780.000000,194623.440000,7708.440000


Country-wise count

In [11]:
CountryCount2 = dataset_df['country'].value_counts().nlargest(50)
print(CountryCount2)

country
Nigeria               572
Côte d'Ivoire         526
Uganda                516
Vietnam               426
Zambia                339
Haiti                 316
South Africa          282
Tanzania              280
Mozambique            277
Zimbabwe              271
Rwanda                268
Ethiopia              101
Guyana                 90
Congo, DRC             84
Ghana                  42
Cameroon               40
Burundi                32
Namibia                24
South Sudan            22
Dominican Republic     18
Swaziland              15
Guatemala               5
Benin                   5
Libya                   4
Malawi                  4
Botswana                3
Angola                  3
Afghanistan             2
Togo                    2
Senegal                 1
Kenya                   1
Name: count, dtype: int64


Country count after value reference removal shows many values discarded as a result. Most notably South African weight and freight cost values where absent from direct value measurements, and highly relied on referenced data. This is also reflected on much of the data according to country wise exploration.

In [12]:
ManLocs2 = dataset_df.groupby('manufacturing site').size().nlargest(5)
print(ManLocs2)

manufacturing site
Aurobindo Unit III, India         1577
Mylan (formerly Matrix) Nashik     887
Hetero Unit III Hyderabad IN       484
Cipla, Goa, India                  353
Strides, Bangalore, India.         315
dtype: int64


When exploring the difference in manufacturing site concerning the indirect reference removal, much of the data is discarded while maintaining the same trend.